# WorkFlow

## Classes

## Load the data

## Test Modelling

## Modelling

**<hr>**

## Classes

In [29]:
import os
import cv2
import torch
import numpy as np

In [30]:
def load_data(img_size=112):
    data = []
    index = -1
    labels = {}
    for directory in os.listdir('./data/'):
        index += 1
        labels[f'./data/{directory}/'] = [index,-1]
    print(len(labels))
    for label in labels:
        for file in os.listdir(label):
            filepath = label + file
            img = cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img,(img_size,img_size))
            img = img / 255.0
            data.append([
                np.array(img),
                np.eye(len(labels))[labels[label][0]]
            ])
            labels[label][1] += 1
    for _ in range(12):
        np.random.shuffle(data)
    print(len(data))
    np.save('./data.npy',data)
    return data

In [31]:
import torch

In [32]:
def other_loading_data_proccess(data):
    X = []
    y = []
    print('going through the data..')
    for d in data:
        X.append(d[0])
        y.append(d[1])
    print('splitting the data')
    VAL_SPLIT = 0.25
    VAL_SPLIT = len(X)*VAL_SPLIT
    VAL_SPLIT = int(VAL_SPLIT)
    X_train = X[:-VAL_SPLIT]
    y_train = y[:-VAL_SPLIT]
    X_test = X[-VAL_SPLIT:]
    y_test = y[-VAL_SPLIT:]
    print('turning data to tensors')
    X_train = torch.from_numpy(np.array(X_train))
    y_train = torch.from_numpy(np.array(y_train))
    X_test = torch.from_numpy(np.array(X_test))
    y_test = torch.from_numpy(np.array(y_test))
    return [X_train,X_test,y_train,y_test]

**<hr>**

## Load the data

In [33]:
REBUILD_DATA = True
if REBUILD_DATA:
    data = load_data()
    np.random.shuffle(data)
    X_train,X_test,y_train,y_test = other_loading_data_proccess(data)

36
2515


/home/indika/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


going through the data..
splitting the data
turning data to tensors


## Test Modelling

In [34]:
import torch
import torch.nn as nn

In [35]:
import torch.nn.functional as F

In [36]:
class Test_Model(nn.Module):
    def __init__(self,output:int=36):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,3)
        self.conv2 = nn.Conv2d(32,64,3)
        self.conv3 = nn.Conv2d(64,128,3)
        self.conv4 = nn.Conv2d(128,256,3)
        self.conv5 = nn.Conv2d(256,384,3)
        self.relu = nn.ReLU()
        self.max_pool2d = F.max_pool2d
        self.fc1 = nn.Linear(384*1*1,32)
        self.fc2 = nn.Linear(32,64)
        self.fc3 = nn.Linear(64,128)
        self.fc4 = nn.Linear(128,256)
        self.fc5 = nn.Linear(256,512)
        self.fc6 = nn.Linear(512,output)
    
    def forward(self,X):
        preds = self.conv1(X)
        preds = self.relu(preds)
        preds = self.max_pool2d(preds,(2,2))
        preds = self.conv2(preds)
        preds = self.relu(preds)
        preds = self.max_pool2d(preds,(2,2))
        preds = self.conv3(preds)
        preds = self.relu(preds)
        preds = self.max_pool2d(preds,(2,2))
        preds = self.conv4(preds)
        preds = self.relu(preds)
        preds = self.max_pool2d(preds,(2,2))
        preds = self.conv5(preds)
        preds = self.relu(preds)
        preds = self.max_pool2d(preds,(2,2))
        preds = preds.view(-1,384*1*1)
        preds = self.fc1(preds)
        preds = self.relu(preds)
        preds = self.fc2(preds)
        preds = self.relu(preds)
        preds = self.fc3(preds)
        preds = self.relu(preds)
        preds = self.fc4(preds)
        preds = self.relu(preds)
        preds = self.fc5(preds)
        preds = self.relu(preds)
        preds = self.fc6(preds)
#         preds = self.relu(preds)
#         return F.softmax(preds,dim=1)
        return preds

In [37]:
device = torch.device('cuda')

In [38]:
model = Test_Model().to(device)

In [39]:
# preds = model(X_test.reshape(-1,1,112,112).float())

In [40]:
# preds[0]

In [41]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.1)
criterion = nn.CrossEntropyLoss()

In [42]:
BATCH_SIZE = 32

In [43]:
EPOCHS = 5

In [44]:
loss_logs = []

In [45]:
from tqdm import tqdm

In [46]:
PROJECT_NAME = "Sign-Language-Recognition"

In [47]:
def test(net,X,y):
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        for i in range(len(X)):
            real_class = torch.argmax(y[i]).to(device)
            net_out = net(X[i].view(-1,1,112,112).to(device).float())
            net_out = net_out[0]
            predictied_class = torch.argmax(net_out)
            if predictied_class == real_class:
                correct += 1
            total += 1
    return round(correct/total,3)

In [48]:
import wandb

In [49]:
import random

In [52]:
index = random.randint(0,29)
print(index)
wandb.init(project=PROJECT_NAME,name='test-CrossEntropyLoss-Adam-0.1')
for _ in tqdm(range(EPOCHS)):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].view(-1,1,112,112).to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        model.to(device)
        preds = model(X_batch.float())
        loss = criterion(preds,y_batch.double())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'loss':loss.item(),'accuracy':test(model,X_train,y_train)*100,'val_accuracy':test(model,X_test,y_test)*100,'pred':torch.argmax(preds[index]),'real':torch.argmax(y_batch[index])})
wandb.finish()

17


  0%|          | 0/5 [00:00<?, ?it/s]


RuntimeError: Expected object of scalar type Long but got scalar type Double for argument #2 'target' in call to _thnn_nll_loss_forward

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

In [ ]:
df = pd.Series(loss_logs)

In [ ]:
df.plot.line(figsize=(12,6))

In [ ]:
test(model,X_test,y_test)

In [ ]:
test(model,X_train,y_train)